In [274]:
from keras import models
from keras import layers
from keras.layers import Input, Dense, Lambda, MaxPooling1D, Conv1D, Flatten, LSTM
import keras.backend as K
from keras.models import Model,Sequential
from keras.regularizers import l2
from keras import Input

#from keras.layers import Input, Conv2D, Lambda, merge, Dense, Flatten,MaxPooling2D

In [275]:

import pickle


with open('hw4_trs.pkl', 'rb') as f:
    data_train = pickle.load(f)
    
    
import numpy as np
import librosa


train_data=[]

for i in range(500):
    S=librosa.stft(data_train[i], n_fft=1024, hop_length=512)
    train_data.append(S)
print(S.shape)
train_data=np.stack(train_data)
print(train_data.shape)

test_data=[]
with open('hw4_tes.pkl', 'rb') as f1:
    data_test = pickle.load(f1)

    
for i in range(200):
    S=librosa.stft(data_test[i], n_fft=1024, hop_length=512)
    test_data.append(S)
print(len(test_data))

test_data=np.stack(test_data)
print(test_data.shape)

(513, 32)
(500, 513, 32)
200
(200, 513, 45)


In [276]:
train_data=np.abs(train_data).transpose((0,2,1))
print(train_data.shape)

(500, 32, 513)


In [277]:
test_data=np.abs(test_data).transpose((0,2,1))
print(test_data.shape)

(200, 45, 513)


In [278]:
print(np.array(l).shape)

(50, 45, 2, 32, 513)


In [285]:
#Siamese Network
# Define the tensors for the two input speech signals
first_In = Input(shape=(None,513))
second_In = Input(shape=(None,513))
    
# Convolutional Neural Network
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=2,activation='relu',strides=1, padding='same',input_shape=(None,513),kernel_initializer='he_normal'))
model.add(MaxPooling1D(pool_size=2,strides=1, padding='valid'))
model.add(Conv1D(filters=32, kernel_size=2,activation='relu',strides=1, padding='same',kernel_initializer='he_normal'))
model.add(MaxPooling1D(pool_size=2,strides=1, padding='valid'))
model.add(Conv1D(filters=16, kernel_size=3,activation='relu',strides=1, padding='same',kernel_initializer='he_normal'))
model.add(MaxPooling1D(pool_size=2,strides=1, padding='valid'))
model.add(Conv1D(filters=32, kernel_size=2, activation='relu', kernel_initializer='he_normal' ))
model.add(LSTM(350))
    
# Generate the encodings (feature vectors) for the two speech signals
encoded_1 = model(first_In)
encoded_2 = model(second_In)
    
 # Add a customized layer to compute the absolute difference between the encodings
L1_layer = Lambda(lambda tensors:K.sum(tensors[0]*tensors[1],axis=-1,keepdims=True)) 
L1_dot = L1_layer([encoded_1, encoded_2])
    
    # Add a dense layer with a sigmoid unit to generate the similarity score
prediction = Dense(1,activation='sigmoid')(L1_dot)
    
    # Connect the inputs with the outputs
siamese_net = Model(inputs=[first_In,second_In],outputs=prediction)
    
    # return the model
siamese_net.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

   

In [286]:
#Generating batches of samples of data
from itertools import combinations
import random
import itertools
def generator(train_data):
    train_data=np.abs(train_data).transpose((0,2,1))
    while True:
        for i in range(0,510,10):
            if i>=len(train_data)-10:
                i=0
            p=np.array(list(combinations(train_data[i:i+10],2)))
            negative_1=list(itertools.product(train_data[i:i+10],train_data[i+10:500]))
            negative_1=random.sample(negative_1,45)
            negative_1=np.array(negative_1)
            data_List=np.concatenate([p, negative_1])
            label=[]
            for i in range(45): 
                
                label.append(1)
            for i in range(45):
                label.append(0)
        
            c = list(zip(data_List, label))
            random.shuffle(c)
            data_List, label = zip(*c)
            data_List=np.array(data_List)
            label=np.array(label)
            data_List=data_List.transpose((1,0,2,3))

            yield [data_List[0],data_List[1]], label[:90]

In [287]:
#Generating batches of samples of data
from itertools import combinations
import random
import itertools
def generator1(test_data):
    test_data=np.abs(test_data).transpose((0,2,1))
    while True:
        for i in range(0,210,10):
            if i>=len(test_data)-10:
                i=0
            p1=np.array(list(combinations(test_data[i:i+10],2)))
            negative_2=list(itertools.product(test_data[i:i+10],test_data[i+10:200]))
            negative_2=random.sample(negative_2,45)
            negative_2=np.array(negative_2)
            data_List1=np.concatenate([p1, negative_2])
            label1=[]
            for i in range(45):
                label1.append(1)
            for i in range(45):
                 label1.append(0)
        
            c1 = list(zip(data_List1, label1))
            random.shuffle(c1)
            data_List1, label1 = zip(*c1)
            data_List1=np.array(data_List1)
            label1=np.array(label1)
            data_List1=data_List1.transpose((1,0,2,3))

            yield [data_List1[0],data_List1[1]], label1[:90]


In [288]:
# Training model
siamese_net.fit_generator(generator(train_data.transpose((0,2,1))),steps_per_epoch = len(train_data)/10, epochs = 50)

Epoch 1/50
50/50 [==============================] - 23s 467ms/step - loss: 0.8356 - acc: 0.5064
Epoch 2/50
50/50 [==============================] - 21s 425ms/step - loss: 0.6875 - acc: 0.5500
Epoch 3/50
50/50 [==============================] - 22s 440ms/step - loss: 0.6763 - acc: 0.6093
Epoch 4/50
50/50 [==============================] - 22s 447ms/step - loss: 0.6790 - acc: 0.6067
Epoch 5/50
50/50 [==============================] - 22s 442ms/step - loss: 0.6688 - acc: 0.6013
Epoch 6/50
50/50 [==============================] - 22s 447ms/step - loss: 0.6580 - acc: 0.6153
Epoch 7/50
50/50 [==============================] - 22s 439ms/step - loss: 0.7312 - acc: 0.6147
Epoch 8/50
50/50 [==============================] - 22s 440ms/step - loss: 0.6424 - acc: 0.6082
Epoch 9/50
50/50 [==============================] - 22s 442ms/step - loss: 0.6167 - acc: 0.6373
Epoch 10/50
50/50 [==============================] - 22s 441ms/step - loss: 0.6049 - acc: 0.6602
Epoch 11/50
50/50 [====================

In [284]:
siamese_net.evaluate_generator(generator1(test_data.transpose((0,2,1))),steps=20)

[0.4929323211312294, 0.7666666716337204]

In [ ]:
siamese_net.evaluate_generator(generator1(test_data.transpose((0,2,1))),steps=20)